In [ ]:
import cv2
import os

input_folder = r"C:\Users\divya_s36xyo5\OneDrive\Desktop\input images"
output_folder = r"C:\Users\divya_s36xyo5\OneDrive\Desktop\output patch"
patch_size = (1024,1024)
stride = 512

os.makedirs(output_folder, exist_ok=True)

# Ensure the input folder exists
if not os.path.exists(input_folder):
    print(f"Error: Input folder '{input_folder}' not found.")
    exit()

# Iterate through files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)

    # Check if the path is a file and ends with ".jpg"
    if os.path.isfile(file_path) and filename.lower().endswith(".jpg"):
       
        # Read the image
        image = cv2.imread(file_path)

        # Patch the image
        for y in range(0, image.shape[0] - patch_size[0] + 1, stride):
            for x in range(0, image.shape[1] - patch_size[1] + 1, stride):
                patch = image[y:y + patch_size[0], x:x + patch_size[1]]
                patch_filename = f"{filename.split('.')[0]}_patch_{y}_{x}.jpg"
                patch_path = os.path.join(output_folder, patch_filename)
                cv2.imwrite(patch_path, patch)

print("Patching completed.")


In [ ]:
pip install imgaug

In [ ]:
import os
import xml.etree.ElementTree as ET
import imgaug.augmenters as iaa
from PIL import Image
import numpy as np
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

def read_annotations(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    boxes = []
    for obj in root.findall('object'):
        box = obj.find('bndbox')
        xmin = int(float(box.find('xmin').text))
        ymin = int(float(box.find('ymin').text))
        xmax = int(float(box.find('xmax').text))
        ymax = int(float(box.find('ymax').text))
        boxes.append(BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax))
    
    return BoundingBoxesOnImage(boxes, shape=(1, 1, 3))


def write_annotations(output_annotation_path, boxes):
    root = ET.Element('annotation')
    obj = ET.SubElement(root, 'object')
    
    for box in boxes.bounding_boxes:
        obj_elem = ET.SubElement(obj, 'bndbox')
        ET.SubElement(obj_elem, 'xmin').text = str(int(box.x1))
        ET.SubElement(obj_elem, 'ymin').text = str(int(box.y1))
        ET.SubElement(obj_elem, 'xmax').text = str(int(box.x2))
        ET.SubElement(obj_elem, 'ymax').text = str(int(box.y2))
    
    tree = ET.ElementTree(root)
    tree.write(output_annotation_path)

def augment_images(images_dir, annotations_dir, output_dir, num_augmented_per_image=5):
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Affine(rotate=(-45, 45)),
        iaa.Multiply((0.8, 1.2)),
        iaa.GaussianBlur(sigma=(0.0, 1.0))
    ])

    os.makedirs(output_dir, exist_ok=True)

    for img_file in os.listdir(os.path.join(images_dir)):
        img_path = os.path.join(images_dir, img_file)
        annotation_path = os.path.join(annotations_dir, img_file.replace('.jpg', '.xml'))

        # Read image and annotations
        img = np.array(Image.open(img_path))
        boxes = read_annotations(annotation_path)

        # Augment images and annotations
        for i in range(num_augmented_per_image):
            augmented_img, augmented_boxes = seq(image=img, bounding_boxes=boxes)
            augmented_img_path = os.path.join(output_dir, f"{img_file.split('.')[0]}_aug_{i}.jpg")
            augmented_annotation_path = os.path.join(output_dir, f"{img_file.split('.')[0]}_aug_{i}.xml")

            Image.fromarray(augmented_img).save(augmented_img_path)
            write_annotations(augmented_annotation_path, augmented_boxes)

if __name__ == "__main__":
    original_images_dir = "Original_Dataset/images"
    original_annotations_dir = "Original_Dataset/annotations"
    augmented_output_dir = "Augmented_Dataset"

    augment_images(original_images_dir, original_annotations_dir, augmented_output_dir)


In [ ]:
import shutil
import os

# Define paths to original and augmented datasets
original_dataset_path = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\Original_Dataset"
augmented_dataset_path = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\Augmented_Dataset"
merged_dataset_path = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\Merged_Dataset_New"

# Check if the merged dataset directory is a directory
if not os.path.isdir(merged_dataset_path):
    # Create a new directory for the merged dataset
    os.makedirs(merged_dataset_path)
    print("Merged dataset directory created.")
else:
    print("Merged dataset directory already exists.")

# Copy images from the original dataset
for root, dirs, files in os.walk(original_dataset_path):
    for file in files:
        src_path = os.path.join(root, file)
        dst_path = os.path.join(merged_dataset_path, file)
        shutil.copy2(src_path, dst_path)

# Copy images from the augmented dataset
shutil.copytree(augmented_dataset_path, os.path.join(merged_dataset_path, "augmented"))

In [ ]:
import os
import shutil

def organize_images_and_annotations(Merged_Dataset_New, Organized_Dataset):
    # Create target folders for images and annotations
    images_folder = os.path.join(Organized_Dataset, "images")
    annotations_folder = os.path.join(Organized_Dataset, "annotations")

    os.makedirs(images_folder, exist_ok=True)
    os.makedirs(annotations_folder, exist_ok=True)

    # Iterate through the dataset folder
    for filename in os.listdir(Merged_Dataset_New):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Copy images to the "images" folder
            shutil.copy2(os.path.join(Merged_Dataset_New, filename), os.path.join(images_folder, filename))
            
            # Assume corresponding annotations have the same name but with a different extension (e.g., .xml)
            annotation_filename = os.path.splitext(filename)[0] + '.xml'
            
            # Check if the corresponding annotation file exists
            if os.path.exists(os.path.join(Merged_Dataset_New, annotation_filename)):
                # Copy annotations to the "annotations" folder
                shutil.copy2(os.path.join(Merged_Dataset_New, annotation_filename), os.path.join(annotations_folder, annotation_filename))
            else:
                print(f"Warning: No annotation file found for {filename}")

# Example usage
Merged_Dataset_New = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\Merged_Dataset_New"
Organized_Dataset = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\Organized_Dataset"

organize_images_and_annotations(Merged_Dataset_New, Organized_Dataset)


In [ ]:
import os
from sklearn.model_selection import train_test_split
import shutil

def split_dataset(images_folder, annotations_folder, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2, random_seed=42):
    # Get lists of image files
    image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]

    # Split the dataset
    train_files, test_val_files = train_test_split(image_files, test_size=(val_ratio + test_ratio), random_state=random_seed)
    val_files, test_files = train_test_split(test_val_files, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=random_seed)

    # Create directories for train, validation, and test sets
    train_folder = os.path.join(images_folder, "train")
    val_folder = os.path.join(images_folder, "val")
    test_folder = os.path.join(images_folder, "test")

    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # Copy images to respective folders
    for file in train_files:
        shutil.copy2(os.path.join(images_folder, file), os.path.join(train_folder, file))
        shutil.copy2(os.path.join(annotations_folder, file.replace('.jpg', '.xml')), os.path.join(train_folder, file.replace('.jpg', '.xml')))

    for file in val_files:
        shutil.copy2(os.path.join(images_folder, file), os.path.join(val_folder, file))
        shutil.copy2(os.path.join(annotations_folder, file.replace('.jpg', '.xml')), os.path.join(val_folder, file.replace('.jpg', '.xml')))

    for file in test_files:
        shutil.copy2(os.path.join(images_folder, file), os.path.join(test_folder, file))
        shutil.copy2(os.path.join(annotations_folder, file.replace('.jpg', '.xml')), os.path.join(test_folder, file.replace('.jpg', '.xml')))

# Example usage
images_folder = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\Organized_Dataset\\images"
annotations_folder = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\Organized_Dataset\\annotations"
split_dataset(images_folder, annotations_folder)


In [ ]:
import tensorflow as tf

PATH_TO_MODEL_DIR = 'path/to/saved_model'
PATH_TO_LABELS = 'path/to/label_map.pbtxt'

model = tf.saved_model.load(PATH_TO_MODEL_DIR)
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [ ]:
# Define the class and its ID
class_info = [{'name': 'sorghum_panicle', 'id': 1}]

# Create label_map.pbtxt content
label_map_content = ''
for item in class_info:
    label_map_content += f"item {{\n  id: {item['id']}\n  name: '{item['name']}'\n}}\n"

# Save to label_map.pbtxt
label_map_path = 'label_map.pbtxt'
with open(label_map_path, 'w') as f:
    f.write(label_map_content)

print(f'Label map file saved to {label_map_path}')


In [ ]:
%run xml_to_csv.py -i Organized_Dataset/annotations -o your_dataset/annotations/train_labels.csv


In [ ]:
import os
import subprocess

# Specify the directory where you want to clone the repository
clone_directory = 'C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop'  # Replace with your desired directory

# Clone the TensorFlow Models repository
command = f'git clone https://github.com/tensorflow/models.git {clone_directory}'
subprocess.run(command, shell=True, check=True)

# Check the contents of the cloned directory
cloned_files = os.listdir(clone_directory)
cloned_files


In [ ]:
import os
import subprocess

# Specify the directory where you want to clone the repository
clone_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop'  # Use forward slashes or double backslashes

# Clone the TensorFlow Models repository
command = f'git clone https://github.com/tensorflow/models.git {clone_directory}/models'
subprocess.run(command, shell=True, check=True)

# Check the contents of the cloned directory
cloned_files = os.listdir(os.path.join(clone_directory, 'models'))
cloned_files


In [ ]:
%run xml_to_csv.py -i Organized_Dataset/annotations -o your_dataset/annotations/train_labels.csv

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(xml_folder, csv_output):
    xml_list = []
    for xml_file in glob.glob(xml_folder + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        size = root.find('size')
        width = int(size.find('width').text)
        height = int(size.find('height').text)
        for member in root.findall('object'):
            class_name = member[0].text
            xmin = round(float(member.find('bndbox').find('xmin').text))
            ymin = round(float(member.find('bndbox').find('ymin').text))
            xmax = round(float(member.find('bndbox').find('xmax').text))
            ymax = round(float(member.find('bndbox').find('ymax').text))
            value = (root.find('filename').text, width, height, class_name, xmin, ymin, xmax, ymax)
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    xml_df.to_csv(csv_output, index=None)
    print(f'Successfully converted XML to CSV. Saved as {csv_output}')

# Example usage
xml_folder = 'C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\Organized_Dataset\\annotations'  # Path to the folder containing XML annotations
csv_output = 'Organized_Dataset/annotations/train_labels.csv'  # Output CSV file
xml_to_csv(xml_folder, csv_output)


In [ ]:
import os
import xml.etree.ElementTree as ET
import imgaug.augmenters as iaa
from PIL import Image
import numpy as np
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

def read_annotations(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    boxes = []
    for obj in root.findall('object'):
        box = obj.find('bndbox')
        xmin = int(float(box.find('xmin').text))
        ymin = int(float(box.find('ymin').text))
        xmax = int(float(box.find('xmax').text))
        ymax = int(float(box.find('ymax').text))
        boxes.append(BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax))

    return BoundingBoxesOnImage(boxes, shape=(1, 1, 3))


def write_annotations(output_annotation_path, boxes, original_size=None):
    root = ET.Element('annotation')

    # Add 'filename' element
    filename_elem = ET.SubElement(root, 'filename')
    filename_elem.text = os.path.basename(output_annotation_path).replace('.xml', '.jpg')

    # Add 'size' element
    size_elem = ET.SubElement(root, 'size')
    width_elem = ET.SubElement(size_elem, 'width')
    height_elem = ET.SubElement(size_elem, 'height')

    # Populate 'width' and 'height' with original or augmented size
    if original_size is not None:
        width_elem.text = str(original_size[0])
        height_elem.text = str(original_size[1])
    else:
        width_elem.text = str(boxes.shape[1])
        height_elem.text = str(boxes.shape[0])

    # Add bounding box elements
    for box in boxes.bounding_boxes:
        obj_elem = ET.SubElement(root, 'object')
        bndbox_elem = ET.SubElement(obj_elem, 'bndbox')
        ET.SubElement(bndbox_elem, 'xmin').text = str(int(box.x1))
        ET.SubElement(bndbox_elem, 'ymin').text = str(int(box.y1))
        ET.SubElement(bndbox_elem, 'xmax').text = str(int(box.x2))
        ET.SubElement(bndbox_elem, 'ymax').text = str(int(box.y2))

    tree = ET.ElementTree(root)
    tree.write(output_annotation_path)

def augment_images(images_dir, annotations_dir, output_dir, num_augmented_per_image=5):
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Affine(rotate=(-45, 45)),
        iaa.Multiply((0.8, 1.2)),
        iaa.GaussianBlur(sigma=(0.0, 1.0))
    ])

    os.makedirs(output_dir, exist_ok=True)

    for img_file in os.listdir(os.path.join(images_dir)):
        img_path = os.path.join(images_dir, img_file)
        annotation_path = os.path.join(annotations_dir, img_file.replace('.jpg', '.xml'))

        # Read image and annotations
        img = np.array(Image.open(img_path))
        original_size = img.shape[:2]
        boxes = read_annotations(annotation_path)

        # Augment images and annotations
        for i in range(num_augmented_per_image):
            augmented_img, augmented_boxes = seq(image=img, bounding_boxes=boxes)
            augmented_img_path = os.path.join(output_dir, f"{img_file.split('.')[0]}_aug_{i}.jpg")
            augmented_annotation_path = os.path.join(output_dir, f"{img_file.split('.')[0]}_aug_{i}.xml")

            Image.fromarray(augmented_img).save(augmented_img_path)
            write_annotations(augmented_annotation_path, augmented_boxes, original_size)

if __name__ == "__main__":
    original_images_dir = "Original_Dataset/images"
    original_annotations_dir = "Original_Dataset/annotations"
    augmented_output_dir = "Augmented_Dataset"

    augment_images(original_images_dir, original_annotations_dir, augmented_output_dir)


In [ ]:
from object_detection.utils import dataset_util
import tensorflow as tf
import pandas as pd

def class_text_to_int(row_label):
    if row_label == 'sorghum_panicle':
        return 1
    else:
        return None

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = tf.io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = [group.xmin / width]
    xmaxs = [group.xmax / width]
    ymins = [group.ymin / height]
    ymaxs = [group.ymax / height]
    classes_text = [group['class'].encode('utf8')]
    classes = [class_text_to_int(group['class'])]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Example usage:
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)
grouped = csv_df.groupby('filename')
tfrecords_output = tf.io.TFRecordWriter(output_path)
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
import sys
!{sys.executable} -m venv myenv
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install tensorflow tensorflow-addons pillow lxml matplotlib


In [ ]:
import os
import subprocess

# Specify the directory where you want to clone the repository
clone_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop'  # Use forward slashes or double backslashes

# Clone the TensorFlow Models repository
command = f'git clone https://github.com/tensorflow/models.git {clone_directory}/models'
subprocess.run(command, shell=True, check=True)

# Check the contents of the cloned directory
cloned_files = os.listdir(os.path.join(clone_directory, 'models'))
cloned_files


In [ ]:
import os
import subprocess

# Specify a different directory for cloning
clone_directory = 'C:\\Users\\divya_s36xyo5\\Desktop\\tensorflow_models'

# Clone the TensorFlow Models repository
command = f'git clone https://github.com/tensorflow/models.git {clone_directory}'
subprocess.run(command, shell=True, check=True)


In [ ]:
import os
import subprocess

# Specify a different directory for cloning
clone_directory = 'C:\\Users\\divya_s36xyo5\\Desktop\\tensorflow_models'

# Clone the TensorFlow Models repository
command = f'git clone https://github.com/tensorflow/models.git {clone_directory}'
subprocess.run(command, shell=True, check=True)


In [ ]:
from object_detection.utils import dataset_util
import tensorflow as tf
import pandas as pd

def class_text_to_int(row_label):
    if row_label == 'sorghum_panicle':
        return 1
    else:
        return None

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = tf.io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = [group.xmin / width]
    xmaxs = [group.xmax / width]
    ymins = [group.ymin / height]
    ymaxs = [group.ymax / height]
    classes_text = [group['class'].encode('utf8')]
    classes = [class_text_to_int(group['class'])]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Example usage:
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)
grouped = csv_df.groupby('filename')
tfrecords_output = tf.io.TFRecordWriter(output_path)
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
# Step 1: Clone the TensorFlow Models repository
!git clone https://github.com/tensorflow/models.git


In [ ]:
# Step 2: Navigate to the Models Directory
import os
os.chdir("models/research")


In [ ]:
# Step 3: Compile Protobufs
!protoc object_detection/protos/*.proto --python_out=.


In [ ]:
import os
import subprocess
import pandas as pd
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util

# Define functions and code for creating TFRecords
# ...

# Example usage:
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)
grouped = csv_df.groupby('filename')
tfrecords_output = tf.io.TFRecordWriter(output_path)
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
pip install --upgrade numpy
pip install --upgrade tensorflow


In [ ]:
# Upgrade NumPy
pip install --upgrade numpy

# Upgrade TensorFlow
pip install --upgrade tensorflow


In [ ]:
# Check NumPy version
pip show numpy

# Check TensorFlow version
pip show tensorflow


In [ ]:
pip uninstall tensorflow
pip install tensorflow


In [ ]:
import os
import io
import pandas as pd
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util

# Function to encode the image
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

# Function to create TFRecord example
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Example usage
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)
grouped = csv_df.groupby('filename')

tfrecords_output = tf.io.TFRecordWriter(output_path)
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from object_detection.utils import dataset_util

In [ ]:
import sys
sys.path.append('C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\models\\research')
sys.path.append('C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\models\\research\\slim')


In [ ]:
from object_detection.utils import dataset_util

In [ ]:
with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename.strip()), 'rb') as fid:


In [ ]:
with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename.strip())), 'rb') as fid:


In [ ]:
with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename.strip())), 'rb') as fid:
    encoded_jpg = fid.read()


In [ ]:

    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename.strip())), 'rb') as fid:
        encoded_jpg = fid.read()

In [ ]:
import os
import io
import pandas as pd
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util

# Function to encode the image
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))


print(f"Image size: {image.size}")

# Function to create TFRecord example
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
    encoded_jpg = fid.read()
encoded_jpg_io = io.BytesIO(encoded_jpg)
image = Image.open(encoded_jpg_io)

print(f"Image size: {image.size}")
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_exampleencoded_jpg_io = io.BytesIO(encoded_jpg)
image = Image.open(encoded_jpg_io)

print(f"Image size: {image.size}")
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


# Example usage
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)
grouped = csv_df.groupby('filename')

tfrecords_output = tf.io.TFRecordWriter(output_path)
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename.strip())), 'rb') as fid:
        encoded_jpg = fid.read()


print(f"Image size: {image.size}")
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)
grouped = csv_df.groupby('filename')

tfrecords_output = tf.io.TFRecordWriter(output_path)
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
import os
import io
import pandas as pd
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util

# Function to encode the image
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

# Function to create TFRecord example
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename.strip())), 'rb') as fid:
        encoded_jpg = fid.read()

    image = Image.open(io.BytesIO(encoded_jpg))
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Example usage
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)
grouped = csv_df.groupby('filename')

tfrecords_output = tf.io.TFRecordWriter(output_path)
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
import os
import io
import pandas as pd
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util

# Function to encode the image
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

# Function to create TFRecord example
def create_tf_example(group, path):
    filename = str(group.filename.iloc[0]).strip()  # Convert to string and strip
    with tf.io.gfile.GFile(os.path.join(path, filename), 'rb') as fid:
        encoded_jpg = fid.read()

    image = Image.open(io.BytesIO(encoded_jpg))
    width, height = image.size

    filename = filename.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Example usage
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)
grouped = csv_df.groupby('filename')

tfrecords_output = tf.io.TFRecordWriter(output_path)
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
import os
import io
import pandas as pd
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util

# Function to encode the image
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

# Function to create TFRecord example
def create_tf_example(group, path):
    filename = str(group.filename.iloc[0]).strip()  # Convert to string and strip
    with tf.io.gfile.GFile(os.path.join(path, filename), 'rb') as fid:
        encoded_jpg = fid.read()

    image = Image.open(io.BytesIO(encoded_jpg))
    width, height = image.size

    filename = filename.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.iterrows():  # Assuming the DataFrame structure here
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Example usage
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)

tfrecords_output = tf.io.TFRecordWriter(output_path)
grouped = csv_df.groupby('filename')
for group in grouped:
    tf_example = create_tf_example(group[1], image_path)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
print(csv_df['class'].unique())


In [ ]:
import os
import io
import pandas as pd
import numpy as np  # Add this import for handling NaN values
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util

# Function to encode the image
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

# Function to create TFRecord example
def create_tf_example(group, path, class_mapping):
    filename = str(group.filename.iloc[0]).strip()  # Convert to string and strip
    with tf.io.gfile.GFile(os.path.join(path, filename), 'rb') as fid:
        encoded_jpg = fid.read()

    image = Image.open(io.BytesIO(encoded_jpg))
    width, height = image.size

    filename = filename.encode('utf8')
    image_format = b'jpeg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.iterrows():
        if pd.notna(row['class']):
            class_label = class_mapping.get(row['class'], None)
            if class_label is not None:
                xmins.append(row['xmin'] / width)
                xmaxs.append(row['xmax'] / width)
                ymins.append(row['ymin'] / height)
                ymaxs.append(row['ymax'] / height)
                classes_text.append(row['class'].encode('utf8'))
                classes.append(class_label)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Example usage
csv_input = 'Organized_Dataset/annotations/train_labels.csv'
output_path = 'Organized_Dataset/annotations/train.record'
image_path = 'Organized_Dataset/images'
csv_df = pd.read_csv(csv_input)

# Create a mapping for class labels
class_mapping = {'sorghum_panicle': 1, 'other_class': 2, 'nan': 0}  # Replace 'other_class' with your desired label for other classes

# Replace 'sorghum_panicle' with NaN and convert the 'class' column to numeric
csv_df['class'] = csv_df['class'].replace('sorghum_panicle', np.nan)
csv_df['class'] = pd.to_numeric(csv_df['class'], errors='coerce')

tfrecords_output = tf.io.TFRecordWriter(output_path)
grouped = csv_df.groupby('filename')
for group in grouped:
    tf_example = create_tf_example(group[1], image_path, class_mapping)
    tfrecords_output.write(tf_example.SerializeToString())
tfrecords_output.close()


In [ ]:
import os
from google.protobuf import compiler
from google.protobuf import descriptor_pb2

def compile_proto(proto_file):
    # Load the .proto file
    with open(proto_file, 'r') as f:
        proto_content = f.read()

    # Create a CodeGeneratorRequest
    request = compiler.plugin_pb2.CodeGeneratorRequest()
    request.files.add(name=proto_file, content=proto_content.encode('utf-8'))

    # Create a CodeGeneratorResponse
    response = compiler.plugin_pb2.CodeGeneratorResponse()

    # Compile the .proto file
    compiler.main(request, response, None)

    # Write the compiled output to files
    for file_proto in response.file:
        output_file = os.path.join('output_directory', file_proto.name)
        with open(output_file, 'wb') as f:
            f.write(file_proto.content)

if __name__ == "__main__":
    # Specify the path to the .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Compile the .proto file
    compile_proto(proto_file_path)


In [ ]:
import os
from google.protobuf import descriptor_pb2
from google.protobuf.compiler import plugin_pb2, code_generator

def compile_proto(proto_file):
    # Load the .proto file
    with open(proto_file, 'r') as f:
        proto_content = f.read()

    # Create a CodeGeneratorRequest
    request = plugin_pb2.CodeGeneratorRequest()
    request.files.add(name=proto_file, content=proto_content.encode('utf-8'))

    # Create a CodeGeneratorResponse
    response = plugin_pb2.CodeGeneratorResponse()

    # Compile the .proto file
    code_generator.run_main(request, response, None)

    # Write the compiled output to files
    for file_proto in response.file:
        output_file = os.path.join('output_directory', file_proto.name)
        with open(output_file, 'wb') as f:
            f.write(file_proto.content)

if __name__ == "__main__":
    # Specify the path to the .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Compile the .proto file
    compile_proto(proto_file_path)


In [ ]:
%%bash

# Run the protoc command to generate protobuf files
protoc object_detection/protos/*.proto --python_out=.


In [ ]:
%system protoc object_detection/protos/*.proto --python_out=.


In [ ]:
import os
os.environ['PATH']


In [ ]:
import os
from google.protobuf import descriptor_pb2
from google.protobuf.compiler import plugin_pb2, code_generator

def compile_proto(proto_file):
    # Load the .proto file
    with open(proto_file, 'r') as f:
        proto_content = f.read()

    # Create a CodeGeneratorRequest
    request = plugin_pb2.CodeGeneratorRequest()
    request.files.add(name=proto_file, content=proto_content.encode('utf-8'))

    # Create a CodeGeneratorResponse
    response = plugin_pb2.CodeGeneratorResponse()

    # Compile the .proto file
    code_generator.run_main(request, response, None)

    # Write the compiled output to files
    for file_proto in response.file:
        output_file = os.path.join('output_directory', file_proto.name)
        with open(output_file, 'wb') as f:
            f.write(file_proto.content)

if __name__ == "__main__":
    # Specify the path to the .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Compile the .proto file
    compile_proto(proto_file_path)


In [ ]:
import os
from google.protobuf import descriptor_pb2
from google.protobuf.compiler.plugin_pb2 import CodeGeneratorRequest, CodeGeneratorResponse
from google.protobuf.compiler import code_generator

def compile_proto(proto_file):
    # Load the .proto file
    with open(proto_file, 'r') as f:
        proto_content = f.read()

    # Create a CodeGeneratorRequest
    request = CodeGeneratorRequest()
    request.files.add(name=proto_file, content=proto_content.encode('utf-8'))

    # Create a CodeGeneratorResponse
    response = CodeGeneratorResponse()

    # Compile the .proto file
    code_generator.run_main(request, response, None)

    # Write the compiled output to files
    for file_proto in response.file:
        output_file = os.path.join('output_directory', file_proto.name)
        with open(output_file, 'wb') as f:
            f.write(file_proto.content)

if __name__ == "__main__":
    # Specify the path to the .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Compile the .proto file
    compile_proto(proto_file_path)


In [ ]:
import os
from google.protobuf import descriptor_pb2
from google.protobuf.compiler.plugin_pb2 import CodeGeneratorRequest, CodeGeneratorResponse

# Import the correct module based on your protobuf version
try:
    from google.protobuf import code_generator as codegen
except ImportError:
    from google.protobuf.compiler import code_generator as codegen

def compile_proto(proto_file):
    # Load the .proto file
    with open(proto_file, 'r') as f:
        proto_content = f.read()

    # Create a CodeGeneratorRequest
    request = CodeGeneratorRequest()
    request.files.add(name=proto_file, content=proto_content.encode('utf-8'))

    # Create a CodeGeneratorResponse
    response = CodeGeneratorResponse()

    # Compile the .proto file
    codegen.run_main(request, response, None)

    # Write the compiled output to files
    for file_proto in response.file:
        output_file = os.path.join('output_directory', file_proto.name)
        with open(output_file, 'wb') as f:
            f.write(file_proto.content)

if __name__ == "__main__":
    # Specify the path to the .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Compile the .proto file
    compile_proto(proto_file_path)


In [ ]:
import subprocess
import os

def compile_proto(proto_file):
    # Specify the output directory for generated files
    output_directory = 'output_directory'
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Run the protoc command to generate Python files
    subprocess.run(['protoc', f'--python_out={output_directory}', proto_file])

    # Optional: Print the result of the protoc command
    result = subprocess.run(['protoc', f'--python_out={output_directory}', proto_file], capture_output=True)
    print(result.stdout.decode())
    print(result.stderr.decode())

if __name__ == "__main__":
    # Specify the path to the .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Compile the .proto file
    compile_proto(proto_file_path)


In [ ]:
result = subprocess.run(['protoc', f'--python_out={output_directory}', proto_file], capture_output=True)
print(result.stdout.decode())
print(result.stderr.decode())


In [ ]:
import subprocess
import os

def compile_proto(proto_file):
    # Specify the output directory for generated files
    output_directory = 'output_directory'
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Specify the proto_path option based on the directory of the .proto file
    proto_path = os.path.dirname(proto_file)

    # Run the protoc command to generate Python files
    subprocess.run(['protoc', f'--proto_path={proto_path}', f'--python_out={output_directory}', proto_file])

    # Optional: Print the result of the protoc command
    result = subprocess.run(['protoc', f'--proto_path={proto_path}', f'--python_out={output_directory}', proto_file], capture_output=True)
    print(result.stdout.decode())
    print(result.stderr.decode())

if __name__ == "__main__":
    # Specify the path to the .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Compile the .proto file
    compile_proto(proto_file_path)


In [ ]:
import subprocess
import os

def compile_proto(proto_file):
    # Specify the output directory for generated files
    output_directory = 'output_directory'
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Specify the proto_path option based on the directory of the .proto file
    proto_path = os.path.dirname(proto_file)

    # Run the protoc command to generate Python files
    subprocess.run(['protoc', f'--proto_path={proto_path}', f'--python_out={output_directory}', proto_file])

    # Optional: Print the result of the protoc command
    result = subprocess.run(['protoc', f'--proto_path={proto_path}', f'--python_out={output_directory}', proto_file], capture_output=True)
    print(result.stdout.decode())
    print(result.stderr.decode())

if __name__ == "__main__":
    # Specify the path to the .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Compile the .proto file
    compile_proto(proto_file_path)


In [ ]:
object_detection/protos/flexible_grid_anchor_generator.proto: File not found.
object_detection/protos/grid_anchor_generator.proto: File not found.
object_detection/protos/multiscale_anchor_generator.proto: File not found.
object_detection/protos/ssd_anchor_generator.proto: File not found.
anchor_generator.proto:6:1: Import "object_detection/protos/flexible_grid_anchor_generator.proto" was not found or had errors.
anchor_generator.proto:7:1: Import "object_detection/protos/grid_anchor_generator.proto" was not found or had errors.
anchor_generator.proto:8:1: Import "object_detection/protos/multiscale_anchor_generator.proto" was not found or had errors.
anchor_generator.proto:9:1: Import "object_detection/protos/ssd_anchor_generator.proto" was not found or had errors.
anchor_generator.proto:15:5: "GridAnchorGenerator" is not defined.
anchor_generator.proto:16:5: "SsdAnchorGenerator" is not defined.
anchor_generator.proto:17:5: "MultiscaleAnchorGenerator" is not defined.
anchor_generator.proto:18:5: "FlexibleGridAnchorGenerator" is not defined.

In [ ]:
import subprocess
import os

def compile_proto(proto_file, proto_paths):
    # Specify the output directory for generated files
    output_directory = 'output_directory'
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Run the protoc command to generate Python files
    subprocess.run(['protoc', f'--proto_path={",".join(proto_paths)}', f'--python_out={output_directory}', proto_file])

    # Optional: Print the result of the protoc command
    result = subprocess.run(['protoc', f'--proto_path={",".join(proto_paths)}', f'--python_out={output_directory}', proto_file], capture_output=True)
    print(result.stdout.decode())
    print(result.stderr.decode())

if __name__ == "__main__":
    # Specify the path to the main .proto file
    proto_file_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/anchor_generator.proto'

    # Specify the paths to the dependency .proto files
    proto_paths = [
        'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/protos/',
        # Add paths for other dependencies if needed
    ]

    # Compile the .proto file
    compile_proto(proto_file_path, proto_paths) 


In [ ]:
import subprocess

# Set the paths to your training script, pipeline configuration, and training directory
train_script_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/legacy/train.py'
pipeline_config_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config'  # Update with your pipeline configuration path
training_dir = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'  # Update with your desired training output directory

# Construct the command to run the training script
command = f'python {train_script_path} --logtostderr --train_dir={training_dir} --pipeline_config_path={pipeline_config_path}'

# Run the command
subprocess.run(command, shell=True)


In [ ]:
import sys
import subprocess

# Add the path to the TensorFlow Models repository to the sys.path
sys.path.append('C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research')

# Command to run the training script
command = f'python C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/legacy/train.py --logtostderr --train_dir=C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output --pipeline_config_path=C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config'

# Run the command
subprocess.run(command, shell=True)


In [ ]:
import sys
import subprocess

# Add the path to the TensorFlow Models repository to the sys.path
sys.path.append('C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research')

# Command to run the training script
command = f'python C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/legacy/train.py --logtostderr --train_dir=C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output --pipeline_config_path=C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config'

# Run the command and capture output and error messages
result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the output and error messages
print("=== Output ===")
print(result.stdout)
print("\n=== Error ===")
print(result.stderr)

# Print the return code
print("\nReturn Code:", result.returncode)


In [ ]:
import sys
import subprocess

# Set the path to the TensorFlow Models repository
models_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research'

# Add the path to the Python path
sys.path.append(models_path)

# Command to run the training script
command = f'python -m object_detection.legacy.train --logtostderr --train_dir=C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output --pipeline_config_path=C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config'

# Run the command and capture output and error messages
result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the output and error messages
print("=== Output ===")
print(result.stdout)
print("\n=== Error ===")
print(result.stderr)

# Print the return code
print("\nReturn Code:", result.returncode)


In [ ]:
# Set the path to the TensorFlow Models repository
models_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research'

# Add the path to the Python path
sys.path.append(models_path)

# Run the training script using the %run magic command
%run -m object_detection.legacy.train --logtostderr --train_dir=C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output --pipeline_config_path=C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config


In [ ]:
# Set the path to the TensorFlow Models repository
models_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research'

# Add the path to the Python path
sys.path.append(models_path)

# Run the training script using the %run magic command
%run -m object_detection.legacy.train --train_dir=C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output --pipeline_config_path=C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config


In [ ]:
# %load C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/legacy/train.py
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""Training executable for detection models.

This executable is used to train DetectionModels. There are two ways of
configuring the training job:

1) A single pipeline_pb2.TrainEvalPipelineConfig configuration file
can be specified by --pipeline_config_path.

Example usage:
    ./train \
        --logtostderr \
        --train_dir=path/to/train_dir \
        --pipeline_config_path=pipeline_config.pbtxt

2) Three configuration files can be provided: a model_pb2.DetectionModel
configuration file to define what type of DetectionModel is being trained, an
input_reader_pb2.InputReader file to specify what training data will be used and
a train_pb2.TrainConfig file to configure training parameters.

Example usage:
    ./train \
        --logtostderr \
        --train_dir=path/to/train_dir \
        --model_config_path=model_config.pbtxt \
        --train_config_path=train_config.pbtxt \
        --input_config_path=train_input_config.pbtxt
"""

import functools
import json
import os
import tensorflow.compat.v1 as tf
from tensorflow.python.util.deprecation import deprecated


from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.legacy import trainer
from object_detection.utils import config_util

tf.logging.set_verbosity(tf.logging.INFO)

flags = tf.app.flags
flags.DEFINE_string('master', '', 'Name of the TensorFlow master to use.')
flags.DEFINE_integer('task', 0, 'task id')
flags.DEFINE_integer('num_clones', 1, 'Number of clones to deploy per worker.')
flags.DEFINE_boolean('clone_on_cpu', False,
                     'Force clones to be deployed on CPU.  Note that even if '
                     'set to False (allowing ops to run on gpu), some ops may '
                     'still be run on the CPU if they have no GPU kernel.')
flags.DEFINE_integer('worker_replicas', 1, 'Number of worker+trainer '
                     'replicas.')
flags.DEFINE_integer('ps_tasks', 0,
                     'Number of parameter server tasks. If None, does not use '
                     'a parameter server.')
flags.DEFINE_string('train_dir', '',
                    'Directory to save the checkpoints and training summaries.')

flags.DEFINE_string('pipeline_config_path', '',
                    'Path to a pipeline_pb2.TrainEvalPipelineConfig config '
                    'file. If provided, other configs are ignored')

flags.DEFINE_string('train_config_path', '',
                    'Path to a train_pb2.TrainConfig config file.')
flags.DEFINE_string('input_config_path', '',
                    'Path to an input_reader_pb2.InputReader config file.')
flags.DEFINE_string('model_config_path', '',
                    'Path to a model_pb2.DetectionModel config file.')

FLAGS = flags.FLAGS


@deprecated(None, 'Use object_detection/model_main.py.')
def main(_):
  assert FLAGS.train_dir, '`train_dir` is missing.'
  if FLAGS.task == 0: tf.gfile.MakeDirs(FLAGS.train_dir)
  if FLAGS.pipeline_config_path:
    configs = config_util.get_configs_from_pipeline_file(
        FLAGS.pipeline_config_path)
    if FLAGS.task == 0:
      tf.gfile.Copy(FLAGS.pipeline_config_path,
                    os.path.join(FLAGS.train_dir, 'pipeline.config'),
                    overwrite=True)
  else:
    configs = config_util.get_configs_from_multiple_files(
        model_config_path=FLAGS.model_config_path,
        train_config_path=FLAGS.train_config_path,
        train_input_config_path=FLAGS.input_config_path)
    if FLAGS.task == 0:
      for name, config in [('model.config', FLAGS.model_config_path),
                           ('train.config', FLAGS.train_config_path),
                           ('input.config', FLAGS.input_config_path)]:
        tf.gfile.Copy(config, os.path.join(FLAGS.train_dir, name),
                      overwrite=True)

  model_config = configs['model']
  train_config = configs['train_config']
  input_config = configs['train_input_config']

  model_fn = functools.partial(
      model_builder.build,
      model_config=model_config,
      is_training=True)

  def get_next(config):
    return dataset_builder.make_initializable_iterator(
        dataset_builder.build(config)).get_next()

  create_input_dict_fn = functools.partial(get_next, input_config)

  env = json.loads(os.environ.get('TF_CONFIG', '{}'))
  cluster_data = env.get('cluster', None)
  cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None
  task_data = env.get('task', None) or {'type': 'master', 'index': 0}
  task_info = type('TaskSpec', (object,), task_data)

  # Parameters for a single worker.
  ps_tasks = 0
  worker_replicas = 1
  worker_job_name = 'lonely_worker'
  task = 0
  is_chief = True
  master = ''

  if cluster_data and 'worker' in cluster_data:
    # Number of total worker replicas include "worker"s and the "master".
    worker_replicas = len(cluster_data['worker']) + 1
  if cluster_data and 'ps' in cluster_data:
    ps_tasks = len(cluster_data['ps'])

  if worker_replicas > 1 and ps_tasks < 1:
    raise ValueError('At least 1 ps task is needed for distributed training.')

  if worker_replicas >= 1 and ps_tasks > 0:
    # Set up distributed training.
    server = tf.train.Server(tf.train.ClusterSpec(cluster), protocol='grpc',
                             job_name=task_info.type,
                             task_index=task_info.index)
    if task_info.type == 'ps':
      server.join()
      return

    worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
    task = task_info.index
    is_chief = (task_info.type == 'master')
    master = server.target

  graph_rewriter_fn = None
  if 'graph_rewriter_config' in configs:
    graph_rewriter_fn = graph_rewriter_builder.build(
        configs['graph_rewriter_config'], is_training=True)

  trainer.train(
      create_input_dict_fn,
      model_fn,
      train_config,
      master,
      task,
      FLAGS.num_clones,
      worker_replicas,
      FLAGS.clone_on_cpu,
      ps_tasks,
      worker_job_name,
      is_chief,
      FLAGS.train_dir,
      graph_hook_fn=graph_rewriter_fn)


if __name__ == '__main__':
  tf.app.run()


In [ ]:
# %load C:/Users/divya_s36xyo5/OneDrive/Desktop/models/research/object_detection/legacy/train.py
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""Training executable for detection models.

This executable is used to train DetectionModels. There are two ways of
configuring the training job:

1) A single pipeline_pb2.TrainEvalPipelineConfig configuration file
can be specified by --pipeline_config_path.

Example usage:
    ./train \
        --logtostderr \
        --train_dir=path/to/train_dir \
        --pipeline_config_path=pipeline_config.pbtxt

2) Three configuration files can be provided: a model_pb2.DetectionModel
configuration file to define what type of DetectionModel is being trained, an
input_reader_pb2.InputReader file to specify what training data will be used and
a train_pb2.TrainConfig file to configure training parameters.

Example usage:
    ./train \
        --logtostderr \
        --train_dir=path/to/train_dir \
        --model_config_path=model_config.pbtxt \
        --train_config_path=train_config.pbtxt \
        --input_config_path=train_input_config.pbtxt
"""

import functools
import json
import os
import tensorflow.compat.v1 as tf
from tensorflow.python.util.deprecation import deprecated


from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.legacy import trainer
from object_detection.utils import config_util

tf.logging.set_verbosity(tf.logging.INFO)

flags = tf.app.flags
flags.DEFINE_string('master', '', 'Name of the TensorFlow master to use.')
flags.DEFINE_integer('task', 0, 'task id')
flags.DEFINE_integer('num_clones', 1, 'Number of clones to deploy per worker.')
flags.DEFINE_boolean('clone_on_cpu', False,
                     'Force clones to be deployed on CPU.  Note that even if '
                     'set to False (allowing ops to run on gpu), some ops may '
                     'still be run on the CPU if they have no GPU kernel.')
flags.DEFINE_integer('worker_replicas', 1, 'Number of worker+trainer '
                     'replicas.')
flags.DEFINE_integer('ps_tasks', 0,
                     'Number of parameter server tasks. If None, does not use '
                     'a parameter server.')
flags.DEFINE_string('train_dir', '',
                    'Directory to save the checkpoints and training summaries.')

flags.DEFINE_string('pipeline_config_path', '',
                    'Path to a pipeline_pb2.TrainEvalPipelineConfig config '
                    'file. If provided, other configs are ignored')

flags.DEFINE_string('train_config_path', '',
                    'Path to a train_pb2.TrainConfig config file.')
flags.DEFINE_string('input_config_path', '',
                    'Path to an input_reader_pb2.InputReader config file.')
flags.DEFINE_string('model_config_path', '',
                    'Path to a model_pb2.DetectionModel config file.')

FLAGS = flags.FLAGS


@deprecated(None, 'Use object_detection/model_main.py.')
def main(_):
  assert FLAGS.train_dir, '`train_dir` is missing.'
  if FLAGS.task == 0: tf.gfile.MakeDirs(FLAGS.train_dir)
  if FLAGS.pipeline_config_path:
    configs = config_util.get_configs_from_pipeline_file(
        FLAGS.pipeline_config_path)
    if FLAGS.task == 0:
      tf.gfile.Copy(FLAGS.pipeline_config_path,
                    os.path.join(FLAGS.train_dir, 'pipeline.config'),
                    overwrite=True)
  else:
    configs = config_util.get_configs_from_multiple_files(
        model_config_path=FLAGS.model_config_path,
        train_config_path=FLAGS.train_config_path,
        train_input_config_path=FLAGS.input_config_path)
    if FLAGS.task == 0:
      for name, config in [('model.config', FLAGS.model_config_path),
                           ('train.config', FLAGS.train_config_path),
                           ('input.config', FLAGS.input_config_path)]:
        tf.gfile.Copy(config, os.path.join(FLAGS.train_dir, name),
                      overwrite=True)

  model_config = configs['model']
  train_config = configs['train_config']
  input_config = configs['train_input_config']

  model_fn = functools.partial(
      model_builder.build,
      model_config=model_config,
      is_training=True)

  def get_next(config):
    return dataset_builder.make_initializable_iterator(
        dataset_builder.build(config)).get_next()

  create_input_dict_fn = functools.partial(get_next, input_config)

  env = json.loads(os.environ.get('TF_CONFIG', '{}'))
  cluster_data = env.get('cluster', None)
  cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None
  task_data = env.get('task', None) or {'type': 'master', 'index': 0}
  task_info = type('TaskSpec', (object,), task_data)

  # Parameters for a single worker.
  ps_tasks = 0
  worker_replicas = 1
  worker_job_name = 'lonely_worker'
  task = 0
  is_chief = True
  master = ''

  if cluster_data and 'worker' in cluster_data:
    # Number of total worker replicas include "worker"s and the "master".
    worker_replicas = len(cluster_data['worker']) + 1
  if cluster_data and 'ps' in cluster_data:
    ps_tasks = len(cluster_data['ps'])

  if worker_replicas > 1 and ps_tasks < 1:
    raise ValueError('At least 1 ps task is needed for distributed training.')

  if worker_replicas >= 1 and ps_tasks > 0:
    # Set up distributed training.
    server = tf.train.Server(tf.train.ClusterSpec(cluster), protocol='grpc',
                             job_name=task_info.type,
                             task_index=task_info.index)
    if task_info.type == 'ps':
      server.join()
      return

    worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
    task = task_info.index
    is_chief = (task_info.type == 'master')
    master = server.target

  graph_rewriter_fn = None
  if 'graph_rewriter_config' in configs:
    graph_rewriter_fn = graph_rewriter_builder.build(
        configs['graph_rewriter_config'], is_training=True)

  trainer.train(
      create_input_dict_fn,
      model_fn,
      train_config,
      master,
      task,
      FLAGS.num_clones,
      worker_replicas,
      FLAGS.clone_on_cpu,
      ps_tasks,
      worker_job_name,
      is_chief,
      FLAGS.train_dir,
      graph_hook_fn=graph_rewriter_fn)


if __name__ == '__main__':
  tf.app.run()


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
%tensorflow_version 2.x


In [ ]:
!pip install --upgrade tensorflow


In [ ]:
# Step 1: Clone the TensorFlow Models repository
import os
import subprocess

# Specify the direc # Replace with your desired directory

# Clone the TensorFlow Models repository





# Move to the models directory
%cd C:/Users/divya_s36xyo5/OneDrive/Desktop/models

# Step 2: Install required dependencies


# Step 4: Confirm the object_detection package is importable
import object_detection

# Step 5: Set paths and configure the model
import functools
import json
import tensorflow.compat.v1 as tf
from tensorflow.python.util.deprecation import deprecated
from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.legacy import trainer
from object_detection.utils import config_util

# Set your paths here
train_dir = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'
pipeline_config_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config'

# Step 6: Configure the model
flags = tf.app.flags
flags.DEFINE_string('train_dir', train_dir, 'Directory to save the checkpoints and training summaries.')
flags.DEFINE_string('pipeline_config_path', pipeline_config_path, 'Path to a pipeline_pb2.TrainEvalPipelineConfig config file.')
FLAGS = flags.FLAGS

@deprecated(None, 'Use object_detection/model_main.py.')
def main(_):
    assert FLAGS.train_dir, '`train_dir` is missing.'
    if FLAGS.task == 0: tf.gfile.MakeDirs(FLAGS.train_dir)
    if FLAGS.pipeline_config_path:
        configs = config_util.get_configs_from_pipeline_file(FLAGS.pipeline_config_path)
        if FLAGS.task == 0:
            tf.gfile.Copy(FLAGS.pipeline_config_path, os.path.join(FLAGS.train_dir, 'pipeline.config'), overwrite=True)
    else:
        configs = config_util.get_configs_from_multiple_files(
            model_config_path=FLAGS.model_config_path,
            train_config_path=FLAGS.train_config_path,
            train_input_config_path=FLAGS.input_config_path)
        if FLAGS.task == 0:
            for name, config in [('model.config', FLAGS.model_config_path),
                                ('train.config', FLAGS.train_config_path),
                                ('input.config', FLAGS.input_config_path)]:
                tf.gfile.Copy(config, os.path.join(FLAGS.train_dir, name), overwrite=True)

    model_config = configs['model']
    train_config = configs['train_config']
    input_config = configs['train_input_config']

    model_fn = functools.partial(
        model_builder.build,
        model_config=model_config,
        is_training=True)

    def get_next(config):
        return dataset_builder.make_initializable_iterator(
            dataset_builder.build(config)).get_next()

    create_input_dict_fn = functools.partial(get_next, input_config)

    env = json.loads(os.environ.get('TF_CONFIG', '{}'))
    cluster_data = env.get('cluster', None)
    cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None
    task_data = env.get('task', None) or {'type': 'master', 'index': 0}
    task_info = type('TaskSpec', (object,), task_data)

    # Parameters for a single worker.
    ps_tasks = 0
    worker_replicas = 1
    worker_job_name = 'lonely_worker'
    task = 0
    is_chief = True
    master = ''

    if cluster_data and 'worker' in cluster_data:
        # Number of total worker replicas include "worker"s and the "master".
        worker_replicas = len(cluster_data['worker']) + 1
    if cluster_data and 'ps' in cluster_data:
        ps_tasks = len(cluster_data['ps'])

    if worker_replicas > 1 and ps_tasks < 1:
        raise ValueError('At least 1 ps task is needed for distributed training.')

    if worker_replicas >= 1 and ps_tasks > 0:
        # Set up distributed training.
        server = tf.train.Server(tf.train.ClusterSpec(cluster), protocol='grpc',
                                job_name=task_info.type,
                                task_index=task_info.index)
        if task_info.type == 'ps':
            server.join()
            return

        worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
    task = task_info.index
    is_chief = (task_info.type == 'master')
    master = server.target

    graph_rewriter_fn = None
    if 'graph_rewriter_config' in configs:
        graph_rewriter_fn = graph_rewriter_builder.build(
            configs['graph_rewriter_config'], is_training=True)

    trainer.train(
        create_input_dict_fn,
        model_fn,
        train_config,
        master,
        task,
        FLAGS.num_clones,
        worker_replicas,
        FLAGS.clone_on_cpu,
        ps_tasks,
        worker_job_name,
        is_chief,
        FLAGS.train_dir,
        graph_hook_fn=graph_rewriter_fn)

if __name__ == '__main__':
    tf.app.run()


In [ ]:
import os
import subprocess
import tensorflow as tf

# Set the path to the TensorFlow Models directory
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'

# Set the path to your custom model directory
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'

# Set the path to your training directory
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Set the path to the pipeline configuration file
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config')

# Change the current working directory to the TensorFlow Models directory
os.chdir(models_directory)

# Train the model using the legacy train script
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir={train_directory} ' \
                f'--pipeline_config_path={pipeline_config_path}'

subprocess.run(train_command, shell=True, check=True)


In [ ]:
import os
import subprocess

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command
subprocess.run(train_command, shell=True)


In [ ]:
import os
import subprocess

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
!pip install tensorflow-object-detection-api


In [ ]:
!git clone https://github.com/tensorflow/models.git


In [ ]:
import os
import subprocess

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Upgrade TensorFlow
!pip install --upgrade tensorflow

# Step 3: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 4: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
os.environ['PYTHONPATH'] += ":/path/to/models/research:/path/to/models/research/slim"


In [ ]:
import os
import subprocess
import tensorflow as tf

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Upgrade TensorFlow
!pip install --upgrade tensorflow

# Step 3: Replace tf.contrib.slim with tf.keras
try:
    slim_example_decoder = tf.contrib.slim.tfexample_decoder
except AttributeError:
    slim_example_decoder = tf.keras.utils.tf_utils.all_variables

# Step 4: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 5: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')



# Step 3: Replace tf.contrib.slim with tf.keras
try:
    slim_example_decoder = tf.compat.v1.estimator.slim.tfexample_decoder
except AttributeError:
    slim_example_decoder = tf.keras.preprocessing.image.ImageDataGenerator  # Replace with the appropriate TensorFlow 2.x equivalent

# Step 4: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 5: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')



# Step 3: Replace tf.contrib.slim with tf.keras
try:
    slim_example_decoder = tf.compat.v1.estimator.slim.tfexample_decoder
except AttributeError:
    slim_example_decoder = tf.keras.preprocessing.image.ImageDataGenerator  # Replace with the appropriate TensorFlow 2.x equivalent

# Step 4: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 5: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf
from tensorflow import keras

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Upgrade TensorFlow
!pip install --upgrade tensorflow

# Step 3: Replace tf.contrib.slim with tf.keras
try:
    slim_example_decoder = tf.contrib.slim.tfexample_decoder
except AttributeError:
    slim_example_decoder = keras.preprocessing.image.ImageDataGenerator  # Replace with the appropriate TensorFlow 2.x equivalent

# Step 4: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 5: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf
from tensorflow import keras

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')



# Step 3: Use TensorFlow 2.x-compatible modules
# Replace the following line with the appropriate TensorFlow 2.x-compatible code
slim_example_decoder = keras.preprocessing.image.ImageDataGenerator

# Step 4: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 5: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf
from tensorflow import keras

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')



# Step 3: Use TensorFlow 2.x-compatible modules
# Replace the following line with the appropriate TensorFlow 2.x-compatible code
slim_example_decoder = keras.preprocessing.image.ImageDataGenerator

# Step 4: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 5: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')


# Step 3: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 4: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf
from object_detection import model_lib_v2

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command using model_lib_v2
train_command = f'python {models_directory}/research/object_detection/model_lib_v2.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Navigate to the models directory
os.chdir(models_directory)

# Step 3: Create and run the training command
train_command = f'python research/object_detection/model_main_tf2.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/model_main.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
!pip install pycocotools

In [ ]:
!pip install tensorflow==1.14

In [ ]:
!git clone https://github.com/tensorflow/models.git


In [ ]:
import os
import subprocess
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/model_main_tf2.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)






In [ ]:
!pip install --upgrade git+https://github.com/tensorflow/models.git


In [ ]:
import object_detection

# P

# List installed modules in the object_detection package
import os
modules_path = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\models\\research\\object_detection"
module_names = [f[:-3] for f in os.listdir(modules_path) if f.endswith('.py')]
print(f'Installed modules in object_detection package: {module_names}')


In [ ]:
from official.vision.image_classification.efficientnet import efficientnet_model


In [ ]:
from official.vision.image_classification.efficientnet import efficientnet_model


In [ ]:
# Step 1: Clone the TensorFlow Models repository
import os
import subprocess

# Specify the directory to clone the TensorFlow Models repository
# Replace with your desired directory
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'

# Clone the TensorFlow Models repository
subprocess.run(['git', 'clone', 'https://github.com/tensorflow/models.git', models_directory])

# Move to the models directory
os.chdir(models_directory)

# Step 2: Install required dependencies
# You might need to install necessary dependencies using:
# pip install -r official/requirements.txt

# Step 4: Confirm the object_detection package is importable
import object_detection

# Step 5: Set paths and configure the model
import functools
import json
import tensorflow.compat.v1 as tf
from tensorflow.python.util.deprecation import deprecated
from object_detection.builders import model_builder
from object_detection.model_lib_v2 import trainer
from object_detection.utils import config_util

# Set your paths here
train_dir = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'
pipeline_config_path = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/pipeline.config'

# Step 6: Configure the model
flags = tf.compat.v1.app.flags
flags.DEFINE_string('train_dir', train_dir, 'Directory to save the checkpoints and training summaries.')
flags.DEFINE_string('pipeline_config_path', pipeline_config_path, 'Path to a pipeline_pb2.TrainEvalPipelineConfig config file.')
FLAGS = flags.FLAGS

@deprecated(None, 'Use object_detection/model_main.py.')
def main(_):
    assert FLAGS.train_dir, '`train_dir` is missing.'
    if FLAGS.task == 0: tf.io.gfile.makedirs(FLAGS.train_dir)
    if FLAGS.pipeline_config_path:
        configs = config_util.get_configs_from_pipeline_file(FLAGS.pipeline_config_path)
        if FLAGS.task == 0:
            tf.io.gfile.copy(FLAGS.pipeline_config_path, os.path.join(FLAGS.train_dir, 'pipeline.config'), overwrite=True)
    else:
        configs = config_util.get_configs_from_multiple_files(
            model_config_path=FLAGS.model_config_path,
            train_config_path=FLAGS.train_config_path,
            train_input_config_path=FLAGS.input_config_path)
        if FLAGS.task == 0:
            for name, config in [('model.config', FLAGS.model_config_path),
                                ('train.config', FLAGS.train_config_path),
                                ('input.config', FLAGS.input_config_path)]:
                tf.io.gfile.copy(config, os.path.join(FLAGS.train_dir, name), overwrite=True)

    model_config = configs['model']
    train_config = configs['train_config']
    input_config = configs['train_input_config']

    model_fn = functools.partial(
        model_builder.build,
        model_config=model_config,
        is_training=True)

    def get_next(config):
        return dataset_builder.make_initializable_iterator(
            dataset_builder.build(config)).get_next()

    create_input_dict_fn = functools.partial(get_next, input_config)

    env = json.loads(os.environ.get('TF_CONFIG', '{}'))
    cluster_data = env.get('cluster', None)
    cluster = tf.distribute.cluster_resolver.TFConfigClusterResolver(cluster_data) if cluster_data else None
    task_data = env.get('task', None) or {'type': 'master', 'index': 0}
    task_info = type('TaskSpec', (object,), task_data)

    # Parameters for a single worker.
    ps_tasks = 0
    worker_replicas = 1
    worker_job_name = 'lonely_worker'
    task = 0
    is_chief = True
    master = ''

    if cluster_data and 'worker' in cluster_data:
        # Number of total worker replicas include "worker"s and the "master".
        worker_replicas = len(cluster_data['worker']) + 1
    if cluster_data and 'ps' in cluster_data:
        ps_tasks = len(cluster_data['ps'])

    if worker_replicas > 1 and ps_tasks < 1:
        raise ValueError('At least 1 ps task is needed for distributed training.')

    if worker_replicas >= 1 and ps_tasks > 0:
        # Set up distributed training.
        server = tf.distribute.Server(cluster, protocol='grpc',
                                job_name=task_info.type,
                                task_index=task_info.index)
        if task_info.type == 'ps':
            server.join()
            return

        worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
    task = task_info.index
    is_chief = (task_info.type == 'master')
    master = server.target

    graph_rewriter_fn = None
    if 'graph_rewriter_config' in configs:
        graph_rewriter_fn = graph_rewriter_builder.build(
            configs['graph_rewriter_config'], is_training=True)

    trainer.train(
        create_input_dict_fn,
        model_fn,
        train_config,
        master,
        task,
        FLAGS.num_clones,
        worker_replicas,
        FLAGS.clone_on_cpu,
        ps_tasks,
        worker_job_name,
        is_chief,
        FLAGS.train_dir,
        graph_hook_fn=graph_rewriter_fn)

if __name__ == '__main__':
    tf.compat.v1.app.run()


In [ ]:
import os
import subprocess
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/model_main_tf2.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)



In [ ]:
import os
import subprocess
import tensorflow as tf
from tensorflow.keras import layers

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/model_main_tf2.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util

# Load label map
label_map_path = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\models\\research\\object_detection\\data\\mscoco_label_map.pbtxt"

# Replace tf.gfile.GFile with tf.io.gfile.GFile
with tf.io.gfile.GFile(label_map_path, 'r') as fid:
    label_map_string = fid.read()

label_map = label_map_util.string_int_label_map_pb2.StringIntLabelMap()
text_format.Merge(label_map_string, label_map)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90)
category_index = label_map_util.create_category_index(categories)


# Load an image for inference
image_path = 'C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\output patch\\DJI_0096_patch_0_0.jpg'
image = tf.image.decode_image(tf.io.read_file(image_path))
image = tf.expand_dims(image, axis=0)

# Perform inference
detections = model(image)

# Visualize the results
image_np = image.numpy().squeeze()
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'][0].numpy(),
    detections['detection_classes'][0].numpy().astype(int),
    detections['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=0.5
)

# Display the result
import matplotlib.pyplot as plt
plt.imshow(image_np)
plt.show()


In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util
from google.protobuf import text_format  # Add this import

# Load label map
label_map_path = "C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\models\\research\\object_detection\\data\\mscoco_label_map.pbtxt"

# Replace tf.gfile.GFile with tf.io.gfile.GFile
with tf.io.gfile.GFile(label_map_path, 'r') as fid:
    label_map_string = fid.read()

label_map = label_map_util.string_int_label_map_pb2.StringIntLabelMap()
text_format.Merge(label_map_string, label_map)  # Fix here
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90)
category_index = label_map_util.create_category_index(categories)


In [ ]:
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
import matplotlib.pyplot as plt

# Load an image for inference
image_path = 'C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\output patch\\DJI_0096_patch_0_0.jpg'
image = tf.image.decode_image(tf.io.read_file(image_path))
image = tf.expand_dims(image, axis=0)

# Perform inference
detections = model(image)

# Convert the image tensor to a NumPy array
image_np = tf.image.convert_image_dtype(image, dtype=tf.uint8).numpy().squeeze()

# Visualize the results
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'][0].numpy(),
    detections['detection_classes'][0].numpy().astype(int),
    detections['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=0.5
)

# Display the result
plt.imshow(image_np)
plt.show()



In [ ]:
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
import matplotlib.pyplot as plt

# Load an image for inference
image_path = 'C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\output patch\\DJI_0096_patch_0_0.jpg'
image = tf.image.decode_image(tf.io.read_file(image_path))
image = tf.expand_dims(image, axis=0)

# Perform inference
detections = model(image)

# Convert the image tensor to a NumPy array
image_np = tf.image.convert_image_dtype(image, dtype=tf.uint8).numpy().squeeze()

# Visualize the results
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'][0].numpy(),
    detections['detection_classes'][0].numpy().astype(int),
    detections['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=0.5
)

# Display the result
plt.imshow(image_np)
plt.show()


In [ ]:
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
import matplotlib.pyplot as plt

# Load an image for inference
image_path = 'C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\output patch\\DJI_0096_patch_0_0.jpg'
image = tf.image.decode_image(tf.io.read_file(image_path))
image = tf.expand_dims(image, axis=0)

# Create a TensorFlow session
with tf.compat.v1.Session() as sess:
    # Perform inference
    detections = sess.run(model(image))

# Convert the image tensor to a NumPy array
image_np = tf.image.convert_image_dtype(image, dtype=tf.uint8).numpy().squeeze()

# Visualize the results
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'][0],
    detections['detection_classes'][0].astype(int),
    detections['detection_scores'][0],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=0.5
)

# Display the result
plt.imshow(image_np)
plt.show()


In [ ]:
import os
import subprocess

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/model_main_tf2.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Print the command
print(train_command)
train_input_reader = {
    'label_map_path': "C:/Users/divya_s36xyo5/OneDrive/Desktop/label_map.pbtxt",
    'tf_record_input_reader': {
        'input_path': "C:/Users/divya_s36xyo5/OneDrive/Desktop/Organized_Dataset/annotations/train.record"
    }
}


print(f"Label Map Path: {os.path.abspath(train_input_reader['label_map_path'])}")
print(f"TFRecord Input Path: {os.path.abspath(train_input_reader['tf_record_input_reader']['input_path'])}")

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
!pip install lvis


In [ ]:
!pip install tensorflow_io


In [ ]:
import sys
sys.path.append('C:\\Users\\divya_s36xyo5\\OneDrive\\Desktop\\models')


In [ ]:
import os
import subprocess

# Set the TensorFlow version to 1.x
%tensorflow_version 1.x

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
import os
import subprocess
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# Set the TensorFlow version to 1.x
tf.compat.v1.disable_v2_behavior()

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Step 1: Modify path separators
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config').replace('\\', '/')

# Step 2: Create the training command
train_command = f'python {models_directory}/research/object_detection/legacy/train.py ' \
                f'--logtostderr ' \
                f'--train_dir="{train_directory}" ' \
                f'--pipeline_config_path="{pipeline_config_path}"'

# Step 3: Run the training command and capture output
result = subprocess.run(train_command, shell=True, capture_output=True, text=True)

# Print the output
print(result.stdout)
print(result.stderr)


In [ ]:
!pip install tf-models-official

In [ ]:
!pip install --upgrade protobuf

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# Set the GPU options if needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Directory paths
models_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/models'
custom_model_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/faster-rcnn/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
train_directory = 'C:/Users/divya_s36xyo5/OneDrive/Desktop/Training_Output'

# Load pipeline config
pipeline_config_path = os.path.join(custom_model_directory, 'pipeline.config')
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)

# Update the config as needed
configs['model'].ssd.num_classes = 90  # Set the number of classes

# Write the updated config back to file
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(pipeline_config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)
pipeline_config.model.ssd.num_classes = 90  # Set the number of classes
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(pipeline_config_path, "wb") as f:
    f.write(config_text)

# Create the dataset and configure the training pipeline
# You need to implement this part based on your dataset and model

# Define your model
model = tf.keras.Sequential([
    # ... Define your model layers here ...
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=num_epochs)

# Save the trained model
saved_model_path = os.path.join(train_directory, 'saved_model')
tf.saved_model.save(model, saved_model_path)

# Optionally, you can convert the saved model to the TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()
tflite_model_path = os.path.join(train_directory, 'model.tflite')
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)


In [ ]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# Set the GPU options if needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [ ]:
!pip install tensorflow


In [ ]:
import os
import tensorflow as tf

# Your code for setting up directories and loading configuration...

# No GPU-specific settings needed for CPU training

# Your model and training code here...
model = tf.keras.Sequential([
    # ... Define your model layers here ...
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=num_epochs)

# Save the trained model
saved_model_path = os.path.join(train_directory, 'saved_model')
tf.saved_model.save(model, saved_model_path)

# Optionally, convert the saved model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()
tflite_model_path = os.path.join(train_directory, 'model.tflite')
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
